# Data Preparation

Focus on school years 2021-2022


In [2]:
# import libraries
import importlib
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import sys

from pathlib import Path

In [3]:
# import other libraries
from helper import load_cde_txt

# check if jcds library is installed
package_name = "jcds"

if importlib.util.find_spec(package_name) is None:
    print(f" '{package_name}' not found. Installing from Github... ")
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "https://github.com/junclemente/jcds.git",
        ]
    )
else:
    print(f" '{package_name}' is already installed.")

from jcds import eda as jeda
from jcds import reports as jrep

 'jcds' is already installed.


In [11]:
data_folder = Path("../data")
ca_doe = Path(data_folder / "raw/ca_doe")
cde = Path(data_folder / "raw/cde")
ca_schls = Path(data_folder / "raw/ca_schls")
civil_rights = Path(data_folder / "raw/civil_rights_data")
raw_data = Path(data_folder / "raw")

# CA Dept of Education - Adjusted Cohort Graduation Rate and Outcome Data (ACGR)

**Adjusted Cohort Graduation Rate and Outcome Data**
Four-year Adjusted Cohort Graduation Rate (ACGR) and Outcome data reported by race/ethnicity, student group, and gender.  
Source: [https://www.cde.ca.gov/ds/ad/filesacgr.asp](https://www.cde.ca.gov/ds/ad/filesacgr.asp)

**Note:** To protect student privacy, data are suppressed (\*) on the data file if the cell size within a selected student population (cohort students) is 10 or less. Additionally, the “Not Reported” race/ethnicity is suppressed, regardless of actual cell size, if the student population for one or more other race/ethnicity groups is suppressed.

Raw Datasets may not be included in the repository due to its size.
The datasets can be downloaded from here: [raw datasets](https://drive.google.com/drive/folders/1OUvpL1nY1uTu2PmjIHJ7wZmY4YgW4onM?usp=sharing)


In [5]:
dataset = "acgr21.txt"
df = load_cde_txt(ca_doe / dataset)
df

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterSchool,DASS,...,SPED Certificate (Count),SPED Certificate (Rate),GED Completer (Count),GED Completer (Rate),Other Transfer (Count),Other Transfer (Rate),Dropout (Count),Dropout (Rate),Still Enrolled (Count),Still Enrolled (Rate)
0,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,All,All,...,57,0.7,0,0.0,15,0.2,449,5.2,305,3.5
1,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,All,All,...,97,1.1,5,0.1,29,0.3,849,9.3,472,5.2
2,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,All,All,...,*,*,*,*,*,*,*,*,*,*
3,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,All,All,...,18,0.4,1,0.0,2,0.0,112,2.5,47,1.1
4,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,All,All,...,27,1.6,0,0.0,3,0.2,172,10.1,139,8.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254933,2020-21,T,00,NaN,NaN,State,NaN,NaN,Yes,Yes,...,0,0.0,1,0.1,6,0.8,273,37.6,269,37.1
254934,2020-21,T,00,NaN,NaN,State,NaN,NaN,Yes,Yes,...,2,0.1,28,0.7,62,1.6,2007,52.1,1056,27.4
254935,2020-21,T,00,NaN,NaN,State,NaN,NaN,Yes,Yes,...,0,0.0,0,0.0,2,1.3,33,22.1,55,36.9
254936,2020-21,T,00,NaN,NaN,State,NaN,NaN,Yes,Yes,...,7,0.0,142,0.6,376,1.6,10306,44.2,6911,29.6


| **Column Name**       | **Description**                                                                                                                                                                                                                                                           | **Values Kept** | **Reason for Filter**                                                                                                              |
| --------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------- | ---------------------------------------------------------------------------------------------------------------------------------- |
| **AggregateLevel**    | Indicates the data aggregation level within the ACGR dataset.<br><br>Possible values:<br>• `T` = State<br>• `C` = County<br>• `D` = District<br>• `S` = School                                                                                                            | `S` (School)    | Focuses analysis on **individual schools**, providing more data points and allowing finer-grained modeling of graduation outcomes. |
| **CharterSchool**     | Indicates whether the record includes data for charter schools.<br><br>Possible values:<br>• `All` = All schools combined<br>• `Y` = Charter only<br>• `N` = Non-charter only                                                                                             | `No` (or `N`)   | Excludes **charter schools** to focus on **traditional public high schools**.                                                      |
| **DASS**              | Dashboard Alternative School Status (DASS) flag.<br><br>Possible values:<br>• `All` = All schools combined<br>• `Y` = DASS only<br>• `N` = Non-DASS only                                                                                                                  | `No` (or `N`)   | Removes **alternative/continuation programs** so graduation rates reflect typical comprehensive high schools.                      |
| **ReportingCategory** | Identifies which subgroup the record represents.<br><br>Examples:<br>• Race/Ethnicity (`RB` = African American, `RH` = Hispanic, etc.)<br>• Gender (`GM` = Male, `GF` = Female)<br>• Program groups (`SD` = Students with Disabilities, etc.)<br>• `TA` = Total Aggregate | `TA`            | Keeps **aggregate totals** for each school (not broken down by subgroup) to simplify modeling.                                     |


In [6]:
# filter dataset
df_filtered = df[
    (df["AggregateLevel"].str.strip() == "S")
    & (df["CharterSchool"].str.strip() == "No")
    & (df["DASS"].str.strip() == "No")
    & (df["ReportingCategory"] == "TA")
]

df_filtered

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterSchool,DASS,...,SPED Certificate (Count),SPED Certificate (Rate),GED Completer (Count),GED Completer (Rate),Other Transfer (Count),Other Transfer (Rate),Dropout (Count),Dropout (Rate),Still Enrolled (Count),Still Enrolled (Rate)
66594,2020-21,S,01,31609,0131755,Alameda,California School for the Blind (State Special...,California School for the Blind,No,No,...,4,36.4,0,0.0,0,0.0,7,63.6,0,0.0
66654,2020-21,S,01,31617,0131763,Alameda,California School for the Deaf-Fremont (State ...,California School for the Deaf-Fremont,No,No,...,2,5.3,0,0.0,0,0.0,1,2.6,11,28.9
66718,2020-21,S,01,61119,0000001,Alameda,Alameda Unified,"Nonpublic, Nonsectarian Schools",No,No,...,*,*,*,*,*,*,*,*,*,*
66782,2020-21,S,01,61119,0106401,Alameda,Alameda Unified,Alameda Science and Technology Institute,No,No,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
66910,2020-21,S,01,61119,0130229,Alameda,Alameda Unified,Alameda High,No,No,...,11,2.8,0,0.0,1,0.3,9,2.3,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254262,2020-21,S,58,72736,0000000,Yuba,Marysville Joint Unified,District Office,No,No,...,0,0.0,0,0.0,0,0.0,30,43.5,0,0.0
254314,2020-21,S,58,72736,0000001,Yuba,Marysville Joint Unified,"Nonpublic, Nonsectarian Schools",No,No,...,*,*,*,*,*,*,*,*,*,*
254426,2020-21,S,58,72736,5830013,Yuba,Marysville Joint Unified,Lindhurst High,No,No,...,0,0.0,0,0.0,2,0.9,14,6.2,13,5.8
254630,2020-21,S,58,72736,5835202,Yuba,Marysville Joint Unified,Marysville High,No,No,...,0,0.0,0,0.0,1,0.5,14,7.0,4,2.0


## Save dataset as pickle file


In [7]:
output_path = Path(data_folder / "acgr21_22_raw.pkl")

df_filtered.to_pickle(output_path)

In [8]:
jrep.data_info(df_filtered, show_columns=True)


SHAPE:
There are 1502 rows and 34 columns (2.97 MB).

DUPLICATES:
There are 0 duplicated rows.

COLUMNS/VARIABLES:
Column dType Summary:
 * object: 34
There are 0 numerical (int/float/bool) variables.
 * Columns: []
There are 34 categorical (nominal/ordinal) variables.
 * Columns: ['AcademicYear', 'AggregateLevel', 'CountyCode', 'DistrictCode', 'SchoolCode', 'CountyName', 'DistrictName', 'SchoolName', 'CharterSchool', 'DASS', 'ReportingCategory', 'CohortStudents', 'Regular HS Diploma Graduates (Count)', 'Regular HS Diploma Graduates (Rate)', "Met UC/CSU Grad Req's (Count)", "Met UC/CSU Grad Req's (Rate)", 'Seal of Biliteracy (Count)', 'Seal of Biliteracy (Rate)', 'Golden State Seal Merit Diploma (Count)', 'Golden State Seal Merit Diploma (Rate', 'CHSPE Completer (Count)', 'CHSPE Completer (Rate)', 'Adult Ed. HS Diploma (Count)', 'Adult Ed. HS Diploma (Rate)', 'SPED Certificate (Count)', 'SPED Certificate (Rate)', 'GED Completer (Count)', 'GED Completer (Rate)', 'Other Transfer (Count)

# CA Dept of Education

[Downloadable Data Files](https://www.cde.ca.gov/ds/ad/downloadabledata.asp)  
Downloadable files about California's K–12 educational system by topic area, including enrollment, assessment and accountability, English learners, foster youth, free or reduced-price meal, graduates and dropouts, and staff data.



## Absenteeism 

The Absenteeism Downloadable Files page provides access to data about student absenteeism, including chronic absenteeism and absenteeism by reason counts and rates, disaggregated by race/ethnicity, gender, student program group, and grade span.

In [12]:
df_chron_abs = load_cde_txt(cde / "chronicabsenteeism21.txt")
# df = pd.read_csv(ca_doe / dataset, sep="\t", dtype=str)
df_chron_abs

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,Reporting Category,ChronicAbsenteeismEligibleCumula,ChronicAbsenteeismCount,ChronicAbsenteeismRate
0,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,Yes,GF,11699,1270,10.9
1,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,All,GF,109029,11268,10.3
2,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,No,GF,97419,10025,10.3
3,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,Yes,GM,11650,1448,12.4
4,2020-21,C,01,NaN,NaN,Alameda,NaN,NaN,No,GM,104393,12414,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263135,2020-21,T,00,NaN,NaN,State,NaN,NaN,All,SS,3745101,724682,19.4
263136,2020-21,T,00,NaN,NaN,State,NaN,NaN,Yes,SS,439770,86323,19.6
263137,2020-21,T,00,NaN,NaN,State,NaN,NaN,All,TA,6072413,866842,14.3
263138,2020-21,T,00,NaN,NaN,State,NaN,NaN,No,TA,5379464,769832,14.3


In [13]:
df_abs_reason = load_cde_txt(cde / "absenteeismreason22-v3.txt")
df_abs_reason

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,DASS,...,Average Days Absent,Total Days Absent,Excused Absences (percent),Unexcused Absences (percent),Out-of-School Suspension Absences (percent),Incomplete Independent Study Absences (percent),Excused Absences (count),Unexcused Absences (count),Out-of-School Suspension Absences (count),Incomplete Independent Study Absences (count)
0,2021-22,T,00,NaN,NaN,State,NaN,NaN,All,All,...,8,30012650,65.1,31,0.6,3.3,19542157.5,9295243.9,178789.9,996458.7
1,2021-22,T,00,NaN,NaN,State,NaN,NaN,All,All,...,34.7,62398169.1,44.6,42,0.7,12.8,27829527.1,26176997,426485.5,7965159.4
2,2021-22,T,00,NaN,NaN,State,NaN,NaN,All,All,...,16.5,44504397.7,51.5,38.1,0.4,10,22918394.3,16950872.4,179887,4455244
3,2021-22,T,00,NaN,NaN,State,NaN,NaN,All,All,...,16.8,47811686.1,51.1,38.7,0.9,9.4,24408640.5,18490919.1,424720.6,4487405.9
4,2021-22,T,00,NaN,NaN,State,NaN,NaN,All,All,...,16,19213037.2,59,34.5,0.1,6.3,11342538.9,6636935.1,19111,1214452.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374737,2021-22,C,58,NaN,NaN,Yuba,NaN,NaN,Yes,Yes,...,*,*,*,*,*,*,*,*,*,*
374738,2021-22,C,58,NaN,NaN,Yuba,NaN,NaN,Yes,Yes,...,15.1,211,0,0,0,100,0,0,0,211
374739,2021-22,C,58,NaN,NaN,Yuba,NaN,NaN,Yes,Yes,...,*,*,*,*,*,*,*,*,*,*
374740,2021-22,C,58,NaN,NaN,Yuba,NaN,NaN,Yes,Yes,...,18.7,2474,0,0,0,100,0,0,0,2474


## Public Schools and Districts

The Public Schools and Districts Downloadable Files page provides access to data files containing general information about California's public schools and districts found in the California School Directory.

In [17]:
df_schooldata = pd.read_excel(cde / "pubschls.xlsx", header=5)

df_schooldata

,CDSCode,NCESDist,NCESSchool,StatusType,County,District,School,Street,StreetAbr,City,...,Virtual,Magnet,YearRoundYN,FederalDFCDistrictID,Latitude,Longitude,AdmFName,AdmLName,LastUpDate,Multilingual
0,01100170000000,0691051,No Data,Active,Alameda,Alameda County Office of Education,No Data,313 West Winton Avenue,313 West Winton Ave.,Hayward,...,No Data,No Data,No Data,No Data,37.658212,-122.09713,Alysse,Castro,2023-03-08,No Data
1,01100170109835,0691051,10546,Closed,Alameda,Alameda County Office of Education,FAME Public Charter,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,...,P,N,N,No Data,37.521436,-121.99391,No Data,No Data,2015-09-01,No Data
2,01100170112607,0691051,10947,Active,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,1515 Webster Street,1515 Webster St.,Oakland,...,N,N,N,0601614,37.804520,-122.26815,Elizabeth,Raji-Greg,2024-05-20,N
3,01100170114363,0691051,12013,Active,Alameda,Alameda County Office of Education,American Indian Public Charter School II,171 12th Street,171 12th St.,Oakland,...,N,N,N,0601880,37.800368,-122.26548,Marco,Menendez,2025-09-08,N
4,01100170118489,0691051,12283,Closed,Alameda,Alameda County Office of Education,Aspire California College Preparatory Academy,2125 Jefferson Avenue,2125 Jefferson Ave.,Berkeley,...,N,N,N,No Data,37.868991,-122.27844,No Data,No Data,2015-07-01,No Data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18400,58727695830039,0642350,No Data,Closed,Yuba,Wheatland Union High,Wheatland Continuation,PO Box 398,PO Box 398,Wheatland,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,1999-06-24,No Data
18401,58727695830070,0642350,No Data,Closed,Yuba,Wheatland Union High,Wheatland Alternative Education,801 Olive Street,801 Olive St.,Wheatland,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,1999-06-24,No Data
18402,58727695830120,0642350,08437,Closed,Yuba,Wheatland Union High,Academy for Career Education Charter,801 Olive Street,801 Olive St.,Wheatland,...,No Data,No Data,N,No Data,39.010399,-121.42927,No Data,No Data,2013-07-02,No Data
18403,58727695838305,0642350,06930,Active,Yuba,Wheatland Union High,Wheatland Union High,1010 Wheatland Road,1010 Wheatland Rd.,Wheatland,...,N,N,N,No Data,38.998968,-121.45497,Brandon,Moore,2024-09-13,N


## Free or Reduced-Price Meal (Student Poverty)

The Free or Reduced-Price Meal Downloadable Files page provides access to data about students who are eligible for Free or Reduced-Price Meals (FRPM).

In [20]:
df_frpm = pd.read_excel(
    cde / "frpm2122_v2.xlsx", sheet_name="FRPM School-Level Data ", header=1
)

df_frpm

,Academic Year,County Code,District Code,School Code,County Name,District Name,School Name,District Type,School Type,Educational \nOption Type,...,Free Meal \nCount \n(K-12),Percent (%) \nEligible Free \n(K-12),FRPM Count \n(K-12),Percent (%) \nEligible FRPM \n(K-12),Enrollment \n(Ages 5-17),Free Meal \nCount \n(Ages 5-17),Percent (%) \nEligible Free \n(Ages 5-17),FRPM Count \n(Ages 5-17),Percent (%) \nEligible FRPM \n(Ages 5-17),CALPADS Fall 1 \nCertification Status
0,2021-2022,1,10017,130419,Alameda,Alameda County Office of Education,Alameda County Community,County Office of Education (COE),County Community,County Community School,...,45,0.789474,47,0.824561,37,29,0.783784,31,0.837838,Y
1,2021-2022,1,10017,130401,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,County Office of Education (COE),Juvenile Court Schools,Juvenile Court School,...,64,1.000000,64,1.000000,56,56,1.000000,56,1.000000,Y
2,2021-2022,1,10017,130625,Alameda,Alameda County Office of Education,Alternatives in Action,County Office of Education (COE),High Schools (Public),Traditional,...,113,0.753333,134,0.893333,129,95,0.736434,116,0.899225,Y
3,2021-2022,1,10017,137448,Alameda,Alameda County Office of Education,Aurum Preparatory Academy,County Office of Education (COE),Junior High Schools (Public),Traditional,...,135,0.823171,148,0.902439,164,135,0.823171,148,0.902439,Y
4,2021-2022,1,10017,123968,Alameda,Alameda County Office of Education,Community School for Creative Education,County Office of Education (COE),Elementary Schools (Public),Traditional,...,162,0.801980,177,0.876238,202,162,0.801980,177,0.876238,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10553,2021-2022,58,72769,0,Yuba,Wheatland Union High,District Office,High School District,NaN,NaN,...,3,1.000000,3,1.000000,0,0,0.000000,0,0.000000,Y
10554,2021-2022,58,72769,133751,Yuba,Wheatland Union High,Edward P. Duplex,High School District,Continuation High Schools,Continuation School,...,32,0.711111,45,1.000000,28,16,0.571429,28,1.000000,Y
10555,2021-2022,58,72769,1,Yuba,Wheatland Union High,"Nonpublic, Nonsectarian Schools",High School District,NaN,NaN,...,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,Y
10556,2021-2022,58,72769,123570,Yuba,Wheatland Union High,Wheatland Community Day High,High School District,District Community Day Schools,Community Day School,...,3,0.600000,4,0.800000,5,3,0.600000,4,0.800000,Y


## CBEDS Data about Schools & Districts

Downloadable data files for information about schools and districts, including estimated number of teacher hires, work visa applications, home-to-school transportation, kindergarten program type, and educational calendar.  
[File Structure](https://www.cde.ca.gov/ds/ad/fscbedsorab19.asp)

In [21]:
df_cbeds = load_cde_txt(cde / "cbedsora21b.txt")

df_cbeds

,Cdscode,CountyName,DistrictName,SchoolName,Description,Level,Section,RowNumber,Value,Year
0,01100170000000,Alameda,Alameda County Office of Education,District Office,Life Science,D,B,10,2.5,2122
1,01100170000000,Alameda,Alameda County Office of Education,District Office,Mathematics,D,B,11,8,2122
2,01100170000000,Alameda,Alameda County Office of Education,District Office,Physical Education,D,B,13,2.5,2122
3,01100170000000,Alameda,Alameda County Office of Education,District Office,Physical Science,D,B,14,3.5,2122
4,01100170000000,Alameda,Alameda County Office of Education,District Office,Reading,D,B,15,4.3,2122
...,...,...,...,...,...,...,...,...,...,...
58706,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,Kindergarten None,S,B,4,True,2122
58707,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,Transitional Kindergarten None,S,B,8,True,2122
58708,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,Traditional,S,D,1,True,2122
58709,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,Start Date,S,D,4,20210811,2122


## Staff Data Files 

The Staff Downloadable Files page provides access to data about certificated and classified staff demographic information, staff assignments, student/staff ratios, and estimated teacher hires.

### Student / Staff Ratio

In [23]:
df_ss_ratio = load_cde_txt(cde / "strat2122.txt")

df_ss_ratio

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,DASS,School Grade Span,TOTAL_ENR_N,TCH_FTE_N,ADM_FTE_N,PSV_FTE_N,OTH_FTE_N,STU_TCH_RATIO,STU_ADM_RATIO,STU_PSV_RATIO,STU_OTH_RATIO
0,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,ALL,5882421,"279,044.9","26,567.8","32,278.4","26,093.8",21.1,221.4,182.2,225.4
1,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,GS_69,873475,"41,544.3","2,822.8","3,157.8","1,974.3",21,309.4,276.6,442.4
2,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,GS_912,1735559,"79,610.7","5,091.4","6,961.3","3,503.0",21.8,340.9,249.3,495.5
3,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,GS_K12,1003277,"49,729.3","11,881.3","15,750.9","10,650.6",20.2,84.4,63.7,94.2
4,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,GS_K6,2270110,"108,160.6","6,772.3","6,408.4","9,965.9",21,335.2,354.2,227.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30238,2021-22,S,58,72751,6118806,Yuba,Wheatland,Wheatland Charter Academy,Y,N,GS_K6,87,4.0,0.5,1.5,0.0,21.8,*,58,*
30239,2021-22,S,58,72769,0000000,Yuba,Wheatland Union High,District Office,N,N,GS_K12,3,0.8,1.0,0.1,0.0,*,3,*,*
30240,2021-22,S,58,72769,0123570,Yuba,Wheatland Union High,Wheatland Community Day High,N,Y,GS_K12,5,0.8,0.1,0.1,0.0,*,*,*,*
30241,2021-22,S,58,72769,0133751,Yuba,Wheatland Union High,Edward P. Duplex,N,Y,GS_912,45,2.6,0.1,0.1,0.0,17.3,*,*,*


### Staff Education 

In [24]:
df_staff_ed = load_cde_txt(cde / "sted2122.txt")

df_staff_ed

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,DASS,...,Staff Gender,Total Staff Count,Associate,Baccalaureate,Baccalaureate Plus,Master,Master Plus,Doctorate,Special (Juris Doctor),None
0,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,...,ALL,26853,64,2602,2626,14143,4601,2332,45,440
1,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,...,GF,17533,55,1711,1713,9302,2993,1450,25,284
2,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,...,GM,9320,9,891,913,4841,1608,882,20,156
3,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,...,ALL,2972,1,264,318,1677,513,146,9,44
4,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,...,GF,1764,1,150,194,980,313,97,6,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360940,2021-22,S,58,10587,0117242,Yuba,Yuba County Office of Education,Yuba Environmental Science Charter Academy,Y,N,...,GF,1,0,0,0,0,1,0,0,0
360941,2021-22,S,58,10587,0117242,Yuba,Yuba County Office of Education,Yuba Environmental Science Charter Academy,Y,N,...,ALL,8,0,4,1,2,1,0,0,0
360942,2021-22,S,58,10587,0117242,Yuba,Yuba County Office of Education,Yuba Environmental Science Charter Academy,Y,N,...,GF,8,0,4,1,2,1,0,0,0
360943,2021-22,S,58,10587,0117242,Yuba,Yuba County Office of Education,Yuba Environmental Science Charter Academy,Y,N,...,ALL,8,0,4,1,2,1,0,0,0


### Staff Experience

In [26]:
df_staff_xp = load_cde_txt(cde / "stex2122.txt")

df_staff_xp

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,DASS,Staff Type,School Grade Span,Staff Gender,Total Staff Count,Average Total Years Experience,Average District Years Experience,Experienced,Inexperienced,First Year,Second Year
0,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,ADM,ALL,ALL,26853,18.0,13.2,25681,1172,657,515
1,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,ADM,ALL,GF,17533,18.0,13.4,16783,750,423,327
2,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,ADM,ALL,GM,9320,18.0,12.9,8898,422,234,188
3,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,ADM,GS_69,ALL,2972,17.0,12.9,2839,133,78,55
4,2021-22,T,NaN,NaN,NaN,NaN,NaN,NaN,ALL,ALL,ADM,GS_69,GF,1764,17.1,13.1,1681,83,48,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334474,2021-22,S,52,71621,0000000,Tehama,Red Bluff Union Elementary,District Office,N,N,OTH,GS_K12,GF,2,1.5,1.5,0,2,1,1
334475,2021-22,S,52,71621,0000000,Tehama,Red Bluff Union Elementary,District Office,N,N,PSV,GS_K12,ALL,2,8.5,8.5,1,1,1,0
334476,2021-22,S,52,71621,0000000,Tehama,Red Bluff Union Elementary,District Office,N,N,PSV,GS_K12,GF,1,1.0,1.0,0,1,1,0
334477,2021-22,S,52,71621,0000000,Tehama,Red Bluff Union Elementary,District Office,N,N,PSV,GS_K12,GM,1,16.0,16.0,1,0,0,0


### Enrollment by School

In [27]:
df_enroll = load_cde_txt(cde / "enr202022-v2.txt")

df_enroll

,ACADEMIC_YEAR,CDS_CODE,COUNTY,DISTRICT,SCHOOL,ENR_TYPE,RACE_ETHNICITY,GENDER,GR_KN,GR_1,...,GR_7,GR_8,UNGR_ELM,GR_9,GR_10,GR_11,GR_12,UNGR_SEC,ENR_TOTAL,ADULT
0,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,0,F,0,0,...,3,0,0,1,0,1,0,0,6,0
1,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,0,M,0,0,...,0,0,0,0,1,1,0,0,3,0
2,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,1,F,0,0,...,0,0,0,1,1,0,1,0,3,0
3,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,2,F,0,0,...,0,0,0,1,0,0,0,0,1,0
4,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,C,2,M,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793375,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,F,0,0,...,0,0,0,35,54,60,43,0,192,0
793376,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,M,0,0,...,0,0,0,61,67,61,63,0,252,0
793377,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,X,0,0,...,0,0,0,0,0,1,0,0,1,0
793378,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,9,F,0,0,...,0,0,0,16,10,14,12,0,52,0
